## Text Summarization

In [9]:
from __future__ import print_function

import os
import numpy as np

from keras.models import Model
from keras.layers import Embedding, Dense, Input
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint

In [10]:
HIDDEN_UNITS = 100
DEFAULT_BATCH_SIZE = 64
VERBOSE = 1
DEFAULT_EPOCHS = 10

In [ ]:
def seq2seq_summarizer(document,
                      num_input_tokens,
                      max_input_seq_length,
                      num_target_tokens,
                      max_target_seq_length,
                      input_word2idx,
                      input_idx2word,
                      target_word2idx,
                      target_idx2word):
    
    # Encoder
    encoder_input = Input(shape=(None,), name="encoder_input")
    encoder_input2embedding = Embedding(input_dim=imput_dict_size, output_dim=HIDDEN_UNITS, input_length=max_input_seq_length, name="encoder_input2embedding")
    embedding2lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, dropout=0.3, name="embedding2lstm")    
    # 
    encoder_outputs, encoder_state_h, encoder_state_c = embedding2lstm(encoder_input2embedding(encoder_input))
    encoder_states = [encoder_state_h, encoder_state_c] 

    # Decoder
    decoder_input = Input(shape=(None, self.num_target_tokens), name="decoder_input")
    decoder_input2lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, dropout=0.3, name="decoder_input2lstm")
    # 
    decoder_outputs, decoder_state_h, decoder_state_c = decoder_input2lstm(decoder_input, initial_state=encoder_states)

    lstm2softmax = Dense(units=self.num_target_tokens, activation="softmax", name="lstm2softmax")
    softmax2output = lstm2softmax(decoder_outputs)

    # Encoder-Decoder Modelling
    model = Model(inputs=[encoder_input, decoder_input], outputs=[softmax2output])
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model = model

    # Encoder Modelling
    self.encoder_model = Model(encoder_input, encoder_states)

    # Decoder Modelling
    ddecoder_state_inputs = [Input(shape=(HIDDEN_UNITS,)), Input(shape=(HIDDEN_UNITS,))]
    decoder_outputs, state_h, state_c = decoder_input2lstm(decoder_input, initial_state=decoder_state_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = lstm2softmax(decoder_outputs)
    self.decoder_model = Model([decoder_input] + decoder_state_inputs, [decoder_outputs] + decoder_states)


In [11]:
class seq2seq_document_summarizer(object):
    

    
    def __init__(self, config):
        self.num_input_tokens = config['num_input_tokens']
        self.max_input_seq_length = config['max_input_seq_length']
        self.num_target_tokens = config['num_target_tokens']
        self.max_target_seq_length = config['max_target_seq_length']
        self.input_word2idx = config['input_word2idx']
        self.input_idx2word = config['input_idx2word']
        self.target_word2idx = config['target_word2idx']
        self.target_idx2word = config['target_idx2word']
        self.config = config

        self.version = 0
        if 'version' in config:
            self.version = config['version']

        # Encoder
        encoder_input = Input(shape=(None,), name="encoder_input")
        encoder_input2embedding = Embedding(input_dim=imput_dict_size, output_dim=HIDDEN_UNITS, input_length=self.max_input_seq_length, name="encoder_input2embedding")
        embedding2lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, dropout=0.3, name="embedding2lstm")
        
        encoder_outputs, encoder_state_h, encoder_state_c = embedding2lstm(encoder_input2embedding(encoder_input))
        encoder_states = [encoder_state_h, encoder_state_c] 
        
        # Decoder
        decoder_input = Input(shape=(None, self.num_target_tokens), name="decoder_input")
        decoder_input2lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, dropout=0.3, name="decoder_input2lstm")
        decoder_outputs, decoder_state_h, decoder_state_c = decoder_input2lstm(decoder_input, initial_state=encoder_states)
        
        lstm2softmax = Dense(units=self.num_target_tokens, activation="softmax", name="lstm2softmax")
        softmax2output = lstm2softmax(decoder_outputs)

        # Encoder-Decoder Modelling
        model = Model(inputs=[encoder_input, decoder_input], outputs=[softmax2output])
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        self.model = model
        
        # Encoder Modelling
        self.encoder_model = Model(encoder_input, encoder_states)
        
        # Decoder Modelling
        ddecoder_state_inputs = [Input(shape=(HIDDEN_UNITS,)), Input(shape=(HIDDEN_UNITS,))]
        decoder_outputs, state_h, state_c = decoder_input2lstm(decoder_input, initial_state=decoder_state_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = lstm2softmax(decoder_outputs)
        self.decoder_model = Model([decoder_input] + decoder_state_inputs, [decoder_outputs] + decoder_states)
        
        # 
        def load_weights(self, weight_file_path):
        if os.path.exists(weight_file_path):
            self.model.load_weights(weight_file_path)
            
            
        def transform_input_text(self, texts):
            temp = []
            for line in texts:
                x = []
                for word in line.lower().split(' '):
                    wid = 1
                    if word in self.input_word2idx:
                        wid = self.input_word2idx[word]
                    x.append(wid)
                    if len(x) >= self.max_input_seq_length:
                        break
                temp.append(x)
            temp = pad_sequences(temp, maxlen=self.max_input_seq_length)

            print(temp.shape)
            return temp
    
    
        def transform_target_encoding(self, texts):
            temp = []
            for line in texts:
                x = []
                line2 = 'START ' + line.lower() + ' END'
                for word in line2.split(' '):
                    x.append(word)
                    if len(x) >= self.max_target_seq_length:
                        break
                temp.append(x)

            temp = np.array(temp)
            print(temp.shape)
            return temp
        
        def generate_batch(self, x_samples, y_samples, batch_size):
        num_batches = len(x_samples) // batch_size
        while True:
            for batchIdx in range(0, num_batches):
                start = batchIdx * batch_size
                end = (batchIdx + 1) * batch_size
                encoder_input_data_batch = pad_sequences(x_samples[start:end], self.max_input_seq_length)
                decoder_target_data_batch = np.zeros(shape=(batch_size, self.max_target_seq_length, self.num_target_tokens))
                decoder_input_data_batch = np.zeros(shape=(batch_size, self.max_target_seq_length, self.num_target_tokens))
                for lineIdx, target_words in enumerate(y_samples[start:end]):
                    for idx, w in enumerate(target_words):
                        w2idx = 0  # default [UNK]
                        if w in self.target_word2idx:
                            w2idx = self.target_word2idx[w]
                        if w2idx != 0:
                            decoder_input_data_batch[lineIdx, idx, w2idx] = 1
                            if idx > 0:
                                decoder_target_data_batch[lineIdx, idx - 1, w2idx] = 1
                yield [encoder_input_data_batch, decoder_input_data_batch], decoder_target_data_batch

    @staticmethod
    def get_weight_file_path(model_dir_path):
        return model_dir_path + '/' + Seq2SeqGloVeSummarizer.model_name + '-weights.h5'

    @staticmethod
    def get_config_file_path(model_dir_path):
        return model_dir_path + '/' + Seq2SeqGloVeSummarizer.model_name + '-config.npy'

    @staticmethod
    def get_architecture_file_path(model_dir_path):
        return model_dir_path + '/' + Seq2SeqGloVeSummarizer.model_name + '-architecture.json'

    def fit(self, Xtrain, Ytrain, Xtest, Ytest, epochs=None, batch_size=None, model_dir_path=None):
        if epochs is None:
            epochs = DEFAULT_EPOCHS
        if model_dir_path is None:
            model_dir_path = './models'
        if batch_size is None:
            batch_size = DEFAULT_BATCH_SIZE

        self.version += 1
        self.config['version'] = self.version
        config_file_path = Seq2SeqGloVeSummarizer.get_config_file_path(model_dir_path)
        weight_file_path = Seq2SeqGloVeSummarizer.get_weight_file_path(model_dir_path)
        checkpoint = ModelCheckpoint(weight_file_path)
        np.save(config_file_path, self.config)
        architecture_file_path = Seq2SeqGloVeSummarizer.get_architecture_file_path(model_dir_path)
        open(architecture_file_path, 'w').write(self.model.to_json())

        Ytrain = self.transform_target_encoding(Ytrain)
        Ytest = self.transform_target_encoding(Ytest)

        Xtrain = self.transform_input_text(Xtrain)
        Xtest = self.transform_input_text(Xtest)

        train_gen = self.generate_batch(Xtrain, Ytrain, batch_size)
        test_gen = self.generate_batch(Xtest, Ytest, batch_size)

        train_num_batches = len(Xtrain) // batch_size
        test_num_batches = len(Xtest) // batch_size

        history = self.model.fit_generator(generator=train_gen, steps_per_epoch=train_num_batches,
                                           epochs=epochs,
                                           verbose=VERBOSE, validation_data=test_gen, validation_steps=test_num_batches,
                                           callbacks=[checkpoint])
        self.model.save_weights(weight_file_path)
        return history

    def summarize(self, input_text):
        input_seq = np.zeros(shape=(1, self.max_input_seq_length, GLOVE_EMBEDDING_SIZE))
        for idx, word in enumerate(input_text.lower().split(' ')):
            if idx >= self.max_input_seq_length:
                break
            emb = self.unknown_emb  # default [UNK]
            if word in self.word2em:
                emb = self.word2em[word]
            input_seq[0, idx, :] = emb
        states_value = self.encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, self.num_target_tokens))
        target_seq[0, 0, self.target_word2idx['START']] = 1
        target_text = ''
        target_text_len = 0
        terminated = False
        while not terminated:
            output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)

            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = self.target_idx2word[sample_token_idx]
            target_text_len += 1

            if sample_word != 'START' and sample_word != 'END':
                target_text += ' ' + sample_word

            if sample_word == 'END' or target_text_len >= self.max_target_seq_length:
                terminated = True

            target_seq = np.zeros((1, 1, self.num_target_tokens))
            target_seq[0, 0, sample_token_idx] = 1

            states_value = [h, c]
        return target_text.strip()

IndentationError: expected an indented block (<ipython-input-11-86142d01c521>, line 53)